# RNN Starter Code


In [ ]:
import numpy as np
import pandas
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F

## Read in the Charlottesville weather data

In [ ]:
df = pandas.read_csv("cho_weather.csv")
data = df.to_numpy()[:, 2:9]
print(data)

In [ ]:
for j in range(data.shape[1]):
    for i in range(data.shape[0]):
        if data[i, j] == "M":
            data[i, j] = data[i - 1, j]
        elif data[i, j] == "T":
            data[i, j] = 0.0
            
data = np.float32(data)

N = data.shape[0]
print(data)

In [ ]:
## The following code scales the data to be in the range [0,1], which can be helpful for neural networks
## We need to make sure to "unscale" our predictions for final results!
scaler = preprocessing.MinMaxScaler(feature_range = (0, 1))
data_scaled = scaler.fit_transform(data)

## Linear prediction model

In [ ]:
class LinearPredictor(nn.Module):
    def __init__(self, window_size, x_size = 1, y_size = 1):
        super(LinearPredictor, self).__init__()
        self.y_size = y_size
        self.window_size = window_size
        self.x_size = x_size
        self.linear = nn.Linear(window_size * x_size, 1)

    ## Here h is a dummy hidden variable, just to make this call the same as an RNN
    def forward(self, x, h):
        y = self.linear(x.reshape(1, self.window_size * self.x_size))
        return y.reshape(self.y_size), h

## RNN model

In [ ]:
class RNN(nn.Module):
    def __init__(self, x_size, h_size, y_size):
        super(RNN, self).__init__()

        self.y_size = y_size

        self.x2h = nn.Linear(x_size, h_size)
        self.h2h = nn.Linear(h_size, h_size)
        self.h2y = nn.Linear(h_size, y_size)

    def forward(self, x, h):
        y = torch.zeros(self.y_size)
        for i in range(x.shape[0]):
            h = F.tanh(self.x2h(x[i]) + self.h2h(h))
            y = self.h2y(h) + x[i, 0]
            ## You might try replacing the line above with the following, which does not do the +x[i,0] (skip connection)
            ## y = self.h2y(h)
        return y.reshape(self.y_size), h

## Training routine, works for LinearPredictor or RNN as the model

In [ ]:
def train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size = 1, scale_factor = 1):
    N = x.shape[0]

    for epoch in range(num_epochs):
        total_loss = 0.0
        for i in range(window_size, N):
            model.zero_grad()
            
            h = torch.zeros(1, h_size)
            y, h = model(x[(i - window_size):i], h)

            loss = criterion(y, targets[i])
            loss.backward()

            optimizer.step()
            
            total_loss += loss.detach()

        # convert to mean loss
        total_loss = total_loss / (N - window_size)

        # "scale_factor" is the amount we scaled the data to be [0,1], we need to undo this here
        print(epoch, ": Training Loss = ", total_loss.item() / scale_factor)

## Running the training

In [ ]:
# Pull out the temperature time series
# You might change the x inputs to include other weather variables
temps = data_scaled[:, 0].reshape(data_scaled.shape[0], 1)
x_size = 1
y_size = 1

### SET YOUR PARAMETERS HERE! ####
###
# learning_rate = ...
# num_epochs = ...
# window_size = ...
# h_size = ...

# model = RNN(x_size, h_size, y_size)
### OR ###
# model = LinearPredictor(window_size)

# Set up your loss function (MAE)
criterion = nn.L1Loss()

# Set up a gradient descent optimizer
# You might also try "SGD" instead of "Adam"
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Define inputs and targets
# You should define "start" and "end" of the training data
# "offset" is how far in the future you want to predict (prediction will be (offset + 1) timepoints in the future)
# start = ...
# end = ...
# offset = ...
L = end - start
x = torch.Tensor(temps[start:end]).reshape(L, x_size)
targets = torch.Tensor(temps[(start + offset):(end + offset)]).reshape(L, 1)

out_y = train(targets, x, model, window_size, optimizer, num_epochs, criterion, h_size, scaler.scale_[0])

## Testing the model

The following code computes the loss on the test data (the final year of temperatures). **Make sure that you don't include the test data in your training!**

In [ ]:
def test(model, inputs, window_size, h_size, scale_fact):
    N = inputs.shape[0]
    x_size = inputs.shape[1]

    model = model.eval()

    start = N - offset - 1 - 8760
    end = N - offset - 1

    L = end - start

    x = torch.Tensor(inputs[start:end]).reshape(L, x_size)
    targets = torch.Tensor(inputs[(start + offset):(end + offset), 0])
    y = torch.zeros(L)

    total_loss = 0.0
    with torch.no_grad():
        for i in range(window_size, L):
            h = torch.zeros(1, h_size)
            y[i], h = model(x[(i - window_size):i, :], h)

            loss = criterion(y[i], targets[i])
            total_loss += loss.detach()

    total_loss = total_loss.item() / (L - window_size) / scale_fact
    return total_loss

In [ ]:
print("MAE on the test data =", test(model, temps, window_size, h_size, scaler.scale_[0]))

## Saving and Loading Trained Models

In [ ]:
# Here is how you save a trained model.
torch.save(model.state_dict(), "your_filename_here.pt")

In [ ]:
# And here is how you would load such a saved model
# Make sure you can reload the model and that it gives the same test results that you expect!
# You have to call the same model creation code "LinearPredictor" or "RNN" to create the model first
model = LinearPredictor(window_size)
model.load_state_dict(torch.load("your_filename_here.pt"))